In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import cv2
import matplotlib.pyplot as plt

In [5]:
def imshow(title, image):
  # convert the image frame BGR to RGB color space and display it
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	plt.imshow(image)
	plt.title(title)
	plt.grid(False)
	plt.show()

In [6]:
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
	"bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
	"dog", "horse", "motorbike", "person", "pottedplant", "sheep",
	"sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size = (len(CLASSES), 3))

In [7]:
# load our serialized pre train model
print("[INFO] loading model")
net = cv2.dnn.readNetFromCaffe("/kaggle/input/image-processing/MobileNetSSD_deploy.prototxt.txt", "/kaggle/input/image-processing/MobileNetSSD_deploy.caffemodel")


In [8]:
image = cv2.imread("/kaggle/input/image-processing/car.jpg")
(h,w) = image.shape[:2]
blob = cv2.dnn.blobFromImage(cv2.resize(image, (300,300)), 0.007843, (300,300), 127.5)

In [9]:
print("[INFO] computing object detections.... ")
net.setInput(blob)
detections = net.forward()


In [10]:
for i in np.arange(0, detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    
    if confidence > 0.2:
        idx = int(detections[0,0, i, 1])
        box = detections[0, 0, i, 3:7]*np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        
        # display the prediction
        label = "{}: {:.2f}%".format(CLASSES[idx], confidence*100)
        print("[INFO] {}".format(label))
        cv2.rectangle(image, (startX, startY), (endX, endY), COLORS[idx], 2)
        
        y = startY-15 if startY-15 > 15 else startY+15
        cv2.putText(image, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)
        
imshow("Output", image)